In [2]:
# Useful starting lines
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
%load_ext autoreload
%autoreload 2

## Load the training data into feature matrix, class labels, and event ids:

In [3]:
from proj1_helpers import *
#Modify DATA_PATH if needed
DATA_TRAIN_PATH = '../../data_project1/train.csv'
y, tX, ids = load_csv_data(DATA_TRAIN_PATH)

In [4]:
print("Training examples: ", tX, " & shape: ")
print("Targets: ", y)
print("Ids: ",ids)
print("Shapes of tX, y & Ids: ", tX.shape, y.shape, ids.shape)

Training examples:  [[ 138.47    51.655   97.827 ...    1.24    -2.475  113.497]
 [ 160.937   68.768  103.235 ... -999.    -999.      46.226]
 [-999.     162.172  125.953 ... -999.    -999.      44.251]
 ...
 [ 105.457   60.526   75.839 ... -999.    -999.      41.992]
 [  94.951   19.362   68.812 ... -999.    -999.       0.   ]
 [-999.      72.756   70.831 ... -999.    -999.       0.   ]]  & shape: 
Targets:  [ 1. -1. -1. ...  1. -1. -1.]
Ids:  [100000 100001 100002 ... 349997 349998 349999]
Shapes of tX, y & Ids:  (250000, 30) (250000,) (250000,)


## Do your thing crazy machine learning thing here :) ...

In [5]:
def compute_loss(y, tx, w, typ):
    '''typ = <LOSS_TYPE>(CAPITAL LETTERS)'''
    loss = 0
    N = len(y)
    if typ == "MSE":
        loss = (1/(2*N))*np.sum(np.square(y - (tx@w)))
        
    elif typ == "MAE":
        loss = (1/(2*N))*np.sum(np.abs(y - (tx@w)))
    return loss

In [6]:
def compute_gradient(y, tx, w):
    '''GRADIENT COMPUTATION'''
    N = len(y)
    e = y - tx@w
    grad = (-1/N) * tx.T @ e
    return grad

In [7]:
def gradient_descent(y, tx, initial_w, max_iters, gamma):
    '''BATCH GRADIENT DESCENT'''
    ws = [initial_w]
    losses = []
    w = initial_w
    for n_iter in range(max_iters):
        loss = compute_loss(y, tx, w, "MSE")
        grad = compute_gradient(y, tx, w)
        w = w - gamma * grad
        ws.append(w)
        losses.append(loss)
        print("Gradient Descent({bi}/{ti}): loss={l}, w0={w0}, w1={w1}".format(
              bi=n_iter, ti=max_iters - 1, l=loss, w0=w[0], w1=w[1]))

    return losses, ws

In [8]:
def batch_iter(y, tx, batch_size, num_batches=1, shuffle=True):
    """
    Generate a minibatch iterator for a dataset.
    Takes as input two iterables (here the output desired values 'y' and the input data 'tx')
    Outputs an iterator which gives mini-batches of `batch_size` matching elements from `y` and `tx`.
    Data can be randomly shuffled to avoid ordering in the original data messing with the randomness of the minibatches.
    Example of use :
    for minibatch_y, minibatch_tx in batch_iter(y, tx, 32):
        <DO-SOMETHING>
    """
    data_size = len(y)

    if shuffle:
        shuffle_indices = np.random.permutation(np.arange(data_size))
        shuffled_y = y[shuffle_indices]
        shuffled_tx = tx[shuffle_indices]
    else:
        shuffled_y = y
        shuffled_tx = tx
    for batch_num in range(num_batches):
        start_index = batch_num * batch_size
        end_index = min((batch_num + 1) * batch_size, data_size)
        if start_index != end_index:
            yield shuffled_y[start_index:end_index], shuffled_tx[start_index:end_index]

In [9]:
def compute_stoch_gradient(y, tx, w):
    '''STOCHASTIC GRADIENT DESCENT GRADIENT COMPUTATION''' 
    N = len(y)
    e = y - tx@w
    grad = (-1/N) * tx.T @ e
    return grad


def stochastic_gradient_descent(y, tx, initial_w, batch_size, max_iters, gamma):
    '''STOCHASTIC GRADIENT DESCENT'''
    losses = []
    ws = [initial_w]
    w = initial_w   
    for it in range(max_iters):       
        for minibatch_y, minibatch_tx in batch_iter(y, tx, 1):
            loss = compute_loss(minibatch_y, minibatch_tx, w, "MSE")
            grad = compute_gradient(minibatch_y, minibatch_tx, w)
            w = w - gamma * grad
            losses.append(loss)
            ws.append(w)
    return losses, ws

In [12]:
def compute_normal_gradient(y, tx):
    '''COMPUTE W_STAR: WEIGHT FOR NORMAL EQUATIONS BY LINEAR EQUATION SOLVER'''
    return np.linalg.solve(tx.T@tx, tx.T@y)
def predict_normal_eq(tx, w_star):
    '''COMPUTE PREDICTION FOR NORMAL EQUATION'''
    return tx.T@w_star

In [11]:
def normal_equation(y, tx, initial_w, max_iters, gamma):
    w_star = compute_normal_gradient(y, tx)
    pred = predict_normal_eq(tx, w_star)
        

## Generate predictions and save ouput in csv format for submission:

In [4]:
DATA_TEST_PATH = '../../data_project1/train.csv' # TODO: download train data and supply path here 
_, tX_test, ids_test = load_csv_data(DATA_TEST_PATH)

In [31]:
OUTPUT_PATH = '../../data_project1/train.csv' # TODO: fill in desired name of output file for submission
y_pred = predict_labels(weights, tX_test)
create_csv_submission(ids_test, y_pred, OUTPUT_PATH)